In [1]:
from sqlalchemy import text
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///db_1.db")

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

[('hello world',)]


Выше мы выполняем два SQL-оператора: оператор «CREATE TABLE» [ 1 ] и параметризованный оператор «INSERT» (синтаксис параметризации мы обсудим позже в разделе «Отправка нескольких параметров »). Чтобы зафиксировать работу, проделанную в нашем блоке, мы вызываем Connection.commit()метод, который фиксирует транзакцию. После этого мы можем продолжить выполнение других SQL-операторов и Connection.commit() снова вызывать их для этих операторов. В SQLAlchemy такой стиль называется « commit as you go» (коммит по мере выполнения ).

In [2]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

Существует также другой стиль фиксации данных. Мы можем заранее объявить наш блок «connect» блоком транзакции. Для этого мы используем метод Engine.begin()для получения соединения, а не Engine.connect()метод . Этот метод будет управлять областью действия Connectionи включать всё внутри транзакции либо с помощью COMMIT в конце, если блок выполнен успешно, либо с помощью ROLLBACK, если возникло исключение. Этот стиль известен как begin once

In [3]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )